# Document Alignment

A helper to generate text passage alignments between a pair of documents, using our own Python port of
[the JavaScript implementation by Mees Gelein](https://github.com/MGelein/text-comparison). The JavaScript 
implementation is based on this work:

> Paul Vierthaler and Mees Gelein, "A BLAST-based, Language-agnostic 
> Text Reuse Algorithm with a MARKUS Implementation and Sequence 
> Alignment Optimized for Large Chinese Corpora," 
> Journal of Cultural Analytics. March 18, 2019. 
> DOI: 10.31235/osf.io/7xpqe

In [3]:
import re
from util.text_alignment import align_text

def clean_text(str):
  ascii_only = re.sub('[^A-Za-z0-9\\n ]+', '', str)
  return re.sub('  +', ' ', ascii_only)

def read_file(f):
  with open(f, 'r') as file:
    return clean_text(file.read())

BARCODE_A = 'Z224214403'
BARCODE_B = 'Z256471804'

INPUT_FILE_A = f'../../../travelogues-corpus/16th_century/books/{BARCODE_A}.txt'
INPUT_FILE_B = f'../../../travelogues-corpus/16th_century/books/{BARCODE_B}.txt'

passages = align_text(read_file(INPUT_FILE_A), read_file(INPUT_FILE_B), 8)
f'Aligned {len(passages)} passages'

Creating dict 0Creating dict 1Expanding 519 shared n-grams


'Aligned 0 passages'

The Gelein implementation produces lots of overlapping alignments. We'll get rid of those in this 
step, by

- removing total overlaps (i.e. passages __contained__ inside other passages)
- merging partial overlaps

We're using an interval tree structure to make this fast.

In [2]:
def merge_passages(passages):

  def merge_two(a, b):
    # Phrase that starts first
    first = a if a['start'] <= b['start'] else b
    second = b if a['start'] <= b['start'] else a

    # second phrase is completely inside first ?
    if (second['end'] <= first['end']):
      return first
    else:
      inner_offset = first['end'] - second['start']

      start = first['start']
      end = second['end']
      text = first['text'] + second['text'][inner_offset:]

      return { 'start': start, 'end': end, 'text': text }

  merged = passages[0]
  for i in range(1, len(passages)):
    a = merge_two(merged['a'], passages[i]['a'])
    b = merge_two(merged['b'], passages[i]['b'])

    merged = { 'a': a, 'b': b }
  
  return merged

In [3]:
from intervaltree import Interval, IntervalTree

t = IntervalTree()

for pair in passages:
  start = pair['a']['start']
  end = pair['a']['end']

  existing_intervals = t[start : end]

  # If intervals already exist at this range, merge & replace
  if (len(existing_intervals) > 0):
    existing_passages = [ i.data for i in existing_intervals ]
    existing_passages.append(pair)
    
    # Merge
    merged = merge_passages(existing_passages)
    
    # Replace
    for i in existing_intervals:
      t.remove(i)

    t[start : end] = merged

  else:
    # No overlaps - just add
    t[start : end] = pair

items = t.items()
items = sorted(items, key=lambda x: x[0])
pruned = [ i.data for i in items ]
f'{len(pruned)} aligned passages left after pruning'

'15 aligned passages left after pruning'

In [4]:
import json

# Write results to file
with open(f'../../results/alignment_{BARCODE_A}_{BARCODE_B}.json', 'w') as outfile:
  json.dump(list(pruned), outfile, indent=2)

In [5]:
# Optional: export the texts in the same form as they were used to generate the 
# alignment (i.e. after cleaning), so that we can render a reading view
with open(f'../../results/{BARCODE_A}_cleaned.txt', 'w') as out_a:
  out_a.write(read_file(INPUT_FILE_A))

with open(f'../../results/{BARCODE_B}_cleaned.txt', 'w') as out_b:
  out_b.write(read_file(INPUT_FILE_B))
